In [ ]:
"In data transformation we write functions for train test split, clean, scale etc"

In [1]:
import os

In [2]:
%pwd

'd:\\MLpipeline\\mlpipeline-example\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\MLpipeline\\mlpipeline-example'

In [ ]:
from dataclasses import dataclass
from pathlib import Path

# mention the entity, which reads it from config.yaml file
@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path

In [7]:
# now create configuration manager

from src.mlProject.constants import *
# whatever things are inside constants import everything

from src.mlProject.utils.common import read_yaml,create_directories

# import Data Transformation Config
from  src.mlProject.entity.config_entity import DataTransformationConfig




class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
        )

        return data_transformation_config


In [8]:
import os
from src.mlProject import logger
from sklearn.model_selection import train_test_split
import pandas as pd

In [9]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config

    def clean_data(self, data: pd.DataFrame) -> pd.DataFrame:
        """
        Perform data cleaning steps here.
        - Handle missing values
        - Remove duplicates
        - Format columns, etc.
        """
        logger.info("Starting data cleaning process")

        # Example cleaning steps
        data.drop_duplicates(inplace=True)  # Remove duplicate rows
        data.dropna(inplace=True)  # Remove rows with missing values
        data.reset_index(drop=True, inplace=True)  # Reset index after cleaning
        
        logger.info("Data cleaning completed")
        logger.info(f"Data shape after cleaning: {data.shape}")

        return data

    
    ## Note: You can add different data transformation techniques such as Scaler, PCA and all
    #You can perform all kinds of EDA in ML cycle here before passing this data to the model

    
    def train_test_spliting(self):
        """performing train test split"""
        logger.info("Loading data for train-test splitting")
        data = pd.read_csv(self.config.data_path)

        # Clean the data
        logger.info("Cleaning the data before splitting")
        data = self.clean_data(data)

        # Split the data into training and test sets. (0.75, 0.25) split.
        train, test = train_test_split(data)

        train.to_csv(os.path.join(self.config.root_dir, "train.csv"),index = False)
        test.to_csv(os.path.join(self.config.root_dir, "test.csv"),index = False)

        logger.info("Splited data into training and test sets")
        logger.info(train.shape)
        logger.info(test.shape)

        print(train.shape)
        print(test.shape)
        
    
    
    # In this case you need to scale the features, you can write a scalng function
    # so first clean, then split, then scale

In [10]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.train_test_spliting()
except Exception as e:
    raise e

# now if you go to artifacts, you will find a directory called data transformation,
# and inside that you will find train.csv and test.csv

# now once this is done, convert this using modular coding
# upgrade config entity, and then upgrade config manager, and then upgrade data transformation

[2024-12-22 14:31:25,768: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-12-22 14:31:25,771: INFO: common: yaml file: params.yaml loaded successfully]
[2024-12-22 14:31:25,778: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-12-22 14:31:25,781: INFO: common: created directory at: artifacts]
[2024-12-22 14:31:25,783: INFO: common: created directory at: artifacts/data_transformation]
[2024-12-22 14:31:25,833: INFO: 1637581648: Splited data into training and test sets]
[2024-12-22 14:31:25,835: INFO: 1637581648: (1199, 12)]
[2024-12-22 14:31:25,836: INFO: 1637581648: (400, 12)]
(1199, 12)
(400, 12)
